In [ ]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 40
N_bound_1 = 20
N_bound_2 = 10

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
bound_t = 0+(1-(0))*lhs(1,N_bound_1)
bound_x = -1+(1-(-1))*lhs(1,N_bound_2)

x = traindata[:,0:1]
t = traindata[:,1:2]

x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()
bound_x = torch.from_numpy(bound_x).float()
bound_t = torch.from_numpy(bound_t).float()

x.requires_grad_()
t.requires_grad_()

###########GPU###########
bound_x = bound_x.cuda()
bound_t = bound_t.cuda()
x = x.cuda()
t = t.cuda()
###########GPU###########

In [ ]:
def func(x):
    return np.sin(np.pi * x[:, 0:1]) * np.exp(-x[:, 1:])

In [ ]:
observe_data = np.vstack((np.linspace(-1, 1, num=10), np.full((10), 1))).T
observe_y = func(observe_data)
observe_data = torch.from_numpy(observe_data).float()
observe_y = torch.from_numpy(observe_y).float()

n=5
np.random.seed(3456)
index_a = np.random.randint(0,10,n,)
observe_data = observe_data[index_a]
observe_y = observe_y[index_a]


###########GPU###########
observe_y = observe_y.cuda()
observe_data = observe_data.cuda()
###########GPU###########

In [ ]:
N_test = 2000
test_data = lb+(la-lb)*lhs(2,N_test)
test_y = func(test_data)
test_data = torch.from_numpy(test_data).float()
test_y = torch.from_numpy(test_y).float()
###########GPU###########
test_data = test_data.cuda()
test_y = test_y.cuda()
###########GPU###########

In [ ]:
def relative_l2(u_pred, u_real):

    l2 = torch.norm(u_real - u_pred, p=2) / torch.norm(u_real, p=2)
    
    return l2.item()  # Convert the result back to a Python float

In [ ]:
C1 = torch.tensor(2.0, requires_grad=True)

## $\text{PINN}^{\S}$

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs1 = NN_H2(2, 32, 3, 1)
PINNs1.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

nIter1 = 10000

loss_all_1 = []
test_loss_1 = []
C1_list = []

loss1_value = 1
it = 0

while  it<7000:
    
    ##### loss_Bi  ######
    E_bound_1 = PINNs1(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1))
    loss_bound_1 = torch.mean(torch.square(E_bound_1-torch.sin(torch.tensor(np.pi)*bound_x)))

    E_bound_2_a = PINNs1(torch.cat((-1*torch.ones_like(bound_t),bound_t),axis=1))
    E_bound_2_b = PINNs1(torch.cat((1*torch.ones_like(bound_t),bound_t),axis=1))
    loss_bound_2 = torch.mean(torch.square(E_bound_2_a))+\
                   torch.mean(torch.square(E_bound_2_b))

    loss_bound = loss_bound_1+loss_bound_2
    
    ##### loss f  ######    
    E_inside = PINNs1(torch.cat((x,t),axis=1))

    d_t = autograd.grad(outputs=E_inside, inputs=t,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_x = autograd.grad(outputs=E_inside, inputs=x,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_xx = autograd.grad(outputs=d_x, inputs=x,
                              grad_outputs=torch.ones_like(d_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  


    loss_f = d_t-C1*d_xx+torch.exp(-t)*(torch.sin(torch.tensor(np.pi)*x)-torch.tensor(np.pi)*torch.tensor(np.pi)*torch.sin(torch.tensor(np.pi)*x))
    loss_f = torch.mean(torch.square(loss_f))

    ##### loss observation  ######        
    E_observation = PINNs1(observe_data)
    loss_observation = torch.mean(torch.square(E_observation-observe_y))

    loss = loss_bound+loss_f+loss_observation

    loss_all_1.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    if (it+1) % 1000 == 0  or it==4999:
        
        print('It:', it, 'Loss:', loss.item())
        print(C1)
        C1_list.append(C1.detach().item())    
            
        N_train = 40
        N_bound_1 = 20
        N_bound_2 = 10

        # x,t
        la = np.array([1,1])
        lb = np.array([-1,0])

        traindata = lb+(la-lb)*lhs(2,N_train)
        bound_t = 0+(1-(0))*lhs(1,N_bound_1)
        bound_x = -1+(1-(-1))*lhs(1,N_bound_2)

        x = traindata[:,0:1]
        t = traindata[:,1:2]

        x = torch.from_numpy(x).float()
        t = torch.from_numpy(t).float()
        bound_x = torch.from_numpy(bound_x).float()
        bound_t = torch.from_numpy(bound_t).float()

        x.requires_grad_()
        t.requires_grad_()      
        ###########GPU###########
        bound_x = bound_x.cuda()
        bound_t = bound_t.cuda()
        x = x.cuda()
        t = t.cuda()
        ###########GPU###########
        
        #########  test_loss NRMSE  #########
        pre_y = PINNs1(test_data)
                 
        test_loss = relative_l2(pre_y,test_y)
        test_loss_1.append(test_loss)
           
    it = it + 1        
loss1_value    

## $\text{GA-PINN}^{\S}$

In [ ]:
N_train = 40
N_bound_1 = 20
N_bound_2 = 10

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
bound_t = 0+(1-(0))*lhs(1,N_bound_1)
bound_x = -1+(1-(-1))*lhs(1,N_bound_2)

x = traindata[:,0:1]
t = traindata[:,1:2]

x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()
bound_x = torch.from_numpy(bound_x).float()
bound_t = torch.from_numpy(bound_t).float()

x.requires_grad_()
t.requires_grad_()

###########GPU###########
bound_x = bound_x.cuda()
bound_t = bound_t.cuda()
x = x.cuda()
t = t.cuda()
###########GPU###########

In [ ]:
C2 = torch.tensor(2.0, requires_grad=True)

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(2, 32, 3, 1)
PINNs2.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs2.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)

optimizer1 = optim.Adam(PINNs2.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C2], 'lr': 0.001})

discriminator = get_discriminator(3, 32, 4, 1)
discriminator.cuda()

import torch.nn.init as init
for name, param in discriminator.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)
        
optimizer2 = optim.Adam(discriminator.parameters(), lr=5e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

nIter1 = 10000

loss_all_2 = []
test_loss_2 = []
C2_list = []

loss1_value = 1
it = 0

while  it<7000:
    
    if it <=500:    
    
        ############ loss D ###########
        pre_H = PINNs2(observe_data)

        d_fake = discriminator(torch.cat((observe_data,pre_H.detach()),1))
        d_real = discriminator(torch.cat((observe_data,observe_y),1))

        loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
        #loss_d = torch.mean(1-d_real)

        optimizer2.zero_grad()
        loss_d.backward()
        optimizer2.step()  

        ####### loss G#######
        pre_H = PINNs2(observe_data)  

        d_fake = discriminator(torch.cat((observe_data,pre_H),1))

        loss_L = torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - observe_y))

        optimizer1.zero_grad()
        loss_L.backward()
        optimizer1.step()      
    
    ##### loss_Bi  ######
    E_bound_1 = PINNs2(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1))
    loss_bound_1 = torch.mean(torch.square(E_bound_1-torch.sin(torch.tensor(np.pi)*bound_x)))

    E_bound_2_a = PINNs2(torch.cat((-1*torch.ones_like(bound_t),bound_t),axis=1))
    E_bound_2_b = PINNs2(torch.cat((1*torch.ones_like(bound_t),bound_t),axis=1))
    loss_bound_2 = torch.mean(torch.square(E_bound_2_a))+\
                   torch.mean(torch.square(E_bound_2_b))
    #                torch.mean(torch.square(E_bound_2_a-E_bound_2_b))
    loss_bound = loss_bound_1+loss_bound_2
    
    ##### loss f  ######    
    E_inside = PINNs2(torch.cat((x,t),axis=1))

    d_t = autograd.grad(outputs=E_inside, inputs=t,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_x = autograd.grad(outputs=E_inside, inputs=x,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_xx = autograd.grad(outputs=d_x, inputs=x,
                              grad_outputs=torch.ones_like(d_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  


    loss_f = d_t-C2*d_xx+torch.exp(-t)*(torch.sin(torch.tensor(np.pi)*x)-torch.tensor(np.pi)*torch.tensor(np.pi)*torch.sin(torch.tensor(np.pi)*x))
    loss_f = torch.mean(torch.square(loss_f))

    ##### loss observation  ######        
    E_observation = PINNs2(observe_data)
    loss_observation = torch.mean(torch.square(E_observation-observe_y))

    loss = loss_bound+loss_f+loss_observation

    loss_all_2.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
        
    if (it+1) % 1000 == 0  or it==4999:
        
        print('It:', it, 'Loss:', loss.item())
        print(C2)
        C2_list.append(C2.detach().item())     
        
        N_train = 40
        N_bound_1 = 20
        N_bound_2 = 10

        # x,t
        la = np.array([1,1])
        lb = np.array([-1,0])

        traindata = lb+(la-lb)*lhs(2,N_train)
        bound_t = 0+(1-(0))*lhs(1,N_bound_1)
        bound_x = -1+(1-(-1))*lhs(1,N_bound_2)

        x = traindata[:,0:1]
        t = traindata[:,1:2]

        x = torch.from_numpy(x).float()
        t = torch.from_numpy(t).float()
        bound_x = torch.from_numpy(bound_x).float()
        bound_t = torch.from_numpy(bound_t).float()

        x.requires_grad_()
        t.requires_grad_()      
        ###########GPU###########
        bound_x = bound_x.cuda()
        bound_t = bound_t.cuda()
        x = x.cuda()
        t = t.cuda()
        ###########GPU###########
        
        #########  test_loss NRMSE  #########
        pre_y = PINNs2(test_data)
                 
        test_loss = relative_l2(pre_y,test_y)
        test_loss_2.append(test_loss)
           
    it = it + 1  
    
loss1_value    

## Results

### Save data

In [ ]:
# np.save('../experimental_data/J2/test_loss_1',test_loss_1)
# np.save('../experimental_data/J2/test_loss_2',test_loss_2)
# np.save('../experimental_data/J2/loss_all_1',loss_all_1)
# np.save('../experimental_data/J2/loss_all_2',loss_all_2)
# np.save('../experimental_data/J2/C1_list',C1_list)
# np.save('../experimental_data/J2/C2_list',C2_list)

### Epochs NRMSE

In [ ]:
print('NRMSE_A',test_loss_1[-1],'NRMSE_B',test_loss_2[-1])